In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
from torchsummary import summary
from torchvision import models
from pycocotools.coco import COCO
import torchvision.transforms as transforms
import json
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger


import os, sys
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2


from models.base_line.model import BaseLine
from models.autoencoders.components import ConvBlock
from models.GANs.model import ConstrastDCGAN

/usr/local/anaconda3/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
class ToyDataset(torch_data.Dataset):
    def __init__(self, len=64):
        super().__init__()
        self.len = len
    
    def __getitem__(self, idx):
        return torch.ones(3, 128, 128), torch.ones(3, 128, 128)*2, torch.zeros(1)

    def __len__(self):
        return self.len

In [3]:
gen_train_dl    = torch_data.DataLoader(torch.ones(8, 3, 128, 128), batch_size=4)
disc_train_dl   = torch_data.DataLoader(torch.ones(8, 3, 64, 64), batch_size=4)
ae_train_dl     = torch_data.DataLoader(ToyDataset(8), batch_size=4)

In [4]:
net = BaseLine(
    ae_out_channels=512, 
    gan_input_channels=512*2*2, 
    gan_noise_dim=512,
    ae_train_dl=ae_train_dl,
    disc_train_dl=disc_train_dl,
    gen_train_dl=gen_train_dl
)

In [5]:
net.forward(*net.example_input_array).shape

torch.Size([3, 3, 64, 64])

In [6]:
real_batch=next(iter(disc_train_dl))
real_batch.shape

torch.Size([4, 3, 64, 64])

In [7]:
fake_batch=net._ae_forward(*next(iter(ae_train_dl)))["union"].flatten(1)
fake_batch.shape

torch.Size([4, 2048])

In [8]:
net.gan._disc_step(
    real=real_batch, 
    fake=fake_batch
)

tensor(0.4533, grad_fn=<AddBackward0>)

In [9]:
net.gan._gen_step(fake_batch)

tensor(0.9546, grad_fn=<BinaryCrossEntropyBackward>)

In [12]:
trainer = pl.Trainer(reload_dataloaders_every_epoch=True, max_epochs=2)
trainer.fit(net)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name               | Type           | Params | In sizes       | Out sizes     
----------------------------------------------------------------------------------------
0 | ae                 | ResUnetAE      | 27.2 M | ?              | ?             
1 | conv_input_decoder | ConvBlock      | 4.7 M  | ?              | ?             
2 | bridge_union       | Sequential     | 4.7 M  | [3, 512, 4, 4] | [3, 512, 2, 2]
3 | gan                | ConstrastDCGAN | 26.5 M | ?              | ?             
----------------------------------------------------------------------------------------
63.1 M    Trainable params
0         Non-trainable params
63.1 M    Total params
252.424   Total estimated model params size (MB)
Epoch 1: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s, loss=51.6, v_num=46]

OSError: [Errno 28] No space left on device

logger = TensorBoardLogger("tb_logs/", name="my_model", log_graph=True)
logger.log_graph(net, net.example_input_array)
%load_ext tensorboard
%tensorboard --logdir tb_logs